In [1]:
# imports / initialisations

from __future__ import unicode_literals
from __future__ import print_function

import numpy
import random
import functools
import numbers

from fractions import Fraction

from antares.core.invariants import Invariants

from lips import Particles
from lips.gaussian_rationals import GaussianRational

### Finite Field Arithmetic

In [2]:
class ModP(int):
    'Integers modulus p, with p prime.'
    
    def __new__(cls, num, p):
        assert type(num) in [int, long] and type(p) in [int,long] , "Non integer modulus."
        self = int.__new__(cls, int(num) % int(p))
        self.p = int(p)
        return self
    
    def __str__(self):
        return "%d %% %d" % (self, self.p)

    def __repr__(self):
        return str(self)

    def __neg__(self):
        return ModP(self.p - int(self), self.p)
    
    def __add__(self, other):
        return ModP(int(self) + int(other), self.p)
    
    def __radd__(self, other):
        return ModP(int(other) + int(self), self.p)
    
    def __sub__(self, other):
        return ModP(int(self) - int(other), self.p)
    
    def __rsub__(self, other):
        return ModP(int(other) - int(self), self.p)
    
    def __mul__(self, other):
        return ModP(int(self) * int(other), self.p)
    
    def __rmul__(self, other):
        return ModP(int(other) * int(self), self.p)
    
    def __div__(self, other):
        if not isinstance(other, ModP):
            other = ModP(other, self.p)
        return self * other._inv()
    
    def __rdiv__(self, other):
        return other * self._inv()
    
    def __pow__(self, other):
        assert(type(other) is int)
        return ModP(int(self) ** int(other), self.p)
    
    def _inv(self):
        'Find multiplicative inverse of self in Z mod p using the extended Euclidean algorithm.'

        rcurr = self.p
        rnext = int(self)
        tcurr = 0
        tnext = 1

        while rnext:
            q = rcurr // rnext
            rcurr, rnext = rnext, rcurr - q * rnext
            tcurr, tnext = tnext, tcurr - q * tnext

        if rcurr != 1:
            raise ValueError("Inverse of {} mod {} does not exist. Are you sure %d is prime?" % (self, self.p, self.p))

        return ModP(tcurr, self.p)

In [3]:
oParticles = Particles(6, rational=True)

In [4]:
p = 2 ** 31 - 1

for oParticle in oParticles:
    oParticle._r_sp_d = numpy.array([ModP(random.randrange(0, p), p), ModP(random.randrange(0, p), p)], dtype=ModP)
    oParticle._r_sp_d.shape = (2, 1)
    oParticle._r_sp_d_to_r_sp_u()
    oParticle._four_mom = numpy.array([None, None, None, None])
    oParticle.l_sp_d = numpy.array([ModP(random.randrange(0, p), p), ModP(random.randrange(0, p), p)], dtype=ModP)

oParticles.fix_mom_cons()

In [5]:
print(oParticles.total_mom)
for i in range(len(oParticles)):
    print(oParticles.ldot(i + 1, i + 1))

[[0 % 2147483647 0 % 2147483647]
 [0 % 2147483647 0 % 2147483647]]
0 % 2147483647
0 % 2147483647
0 % 2147483647
0 % 2147483647
0 % 2147483647
0 % 2147483647


In [6]:
oInvariants = Invariants(6)
for oInv in oInvariants.full:
    assert(type(oParticles.compute(oInv)) == ModP)

In [7]:
# some definitions
zab2_4231 = oParticles.compute("⟨4|(2+3)|1]")
zb_16 = oParticles.compute("[1|6]")
za_23 = oParticles.compute("⟨2|3⟩")
za_34 = oParticles.compute("⟨3|4⟩")
zb_56 = oParticles.compute("[5|6]")
zab2_2165 = oParticles.compute("⟨2|(1+6)|5]")
s_234 = oParticles.compute("s_234")
zab2_6123 = oParticles.compute("⟨6|(1+2)|3]")
za_12 = oParticles.compute("⟨1|2⟩")
za_16 = oParticles.compute("⟨1|6⟩")
zb_34 = oParticles.compute("[3|4]")
zb_45 = oParticles.compute("[4|5]")
s_345 = oParticles.compute("s_345")

In [8]:
mIA6g_pppmmm = zab2_4231 ** 3 / (zb_16 * za_23 * za_34 * zb_56 * zab2_2165 * s_234) - zab2_6123 ** 3 / (za_12 * za_16 * zb_34 * zb_45 * zab2_2165 * s_345)

In [9]:
mIA6g_pppmmm

2083655871 % 2147483647

### p-adic Arithmetic

In [10]:
def to_base(num, p):
    if num < p:
        return (num, )
    else:
        return (num % p, ) + (to_base(num // p, p))

In [11]:
def padicfy(func):
    @functools.wraps(func)
    def wrapper_padicfy(self, other):
        if type(other) is PAdic:
            return func(self, other)
        elif type(other) in [int, long, ModP, numpy.int64]:
            return func(self, PAdic(other, self.p, self.k, 0))
        else:
            print(type(self), ", ", type(other))
            print(self, ", ", other)
            raise Exception
    return wrapper_padicfy

In [12]:
class PAdic(int):
    
    'PAdic Integers, with p prime.'
    
    def __new__(cls, num, p, k, n):
        self = int.__new__(cls, int(num) % int(p ** k))
        self.p = int(p)
        self.k = int(k)
        self.n = int(n)
        return self
    
    @property
    def as_tuple(self):
        return (to_base(int(self), self.p) + tuple([0 for i in range(k)]))[:k]
    
    def __str__(self):
        if self == 0:
            return "O({}^{})".format(self.p, self.n + self.k)
        else:
            return " + ".join(filter(lambda x: x is not None,
                                     ["{}".format(i) if (j == 0 and i != 0) else
                                      "{}*{}".format(i, self.p) if (j == 1 and i != 0) else
                                      "{}*{}^{}".format(i, self.p, j) if (i != 0) else None
                                      for i, j in zip(self.as_tuple, range(self.n, self.n + self.k))])) + " + O({}^{})".format(self.p, self.n + self.k)

    def __repr__(self):
        return str(self)

    @padicfy
    def __add__(self, other):
        if self.n > other.n:
            return other + self
        else:
            return PAdic((int(self) + int(other) * self.p ** (other.n - self.n)) % self.p ** self.k, self.p, self.k, self.n)

    @padicfy
    def __radd__(self, other):
        return other + self
    
    @padicfy
    def __sub__(self, other):
        return self + (- other)
    
    @padicfy
    def __rsub__(self, other):
        return - (self - other)
        
    @padicfy
    def __mul__(self, other):
        return PAdic((int(self) * int(other)) % self.p ** self.k, self.p, self.k, self.n + other.n)
    
    @padicfy
    def __rmul__(self, other):
        return other * self
    
    @padicfy
    def __div__(self, other):
        if other == 0:
            raise ZeroDivisionError
        p_fact = next(i for i, j in enumerate(other.as_tuple) if j != 0)
        return PAdic(self *  ModP(int(other) / other.p ** p_fact, other.p ** other.k)._inv() % self.p ** self.k, self.p, self.k, self.n - other.n - p_fact)
    
    @padicfy
    def __rdiv__(self, other):
        return other / self

    def __neg__(self):
        return PAdic((-1 * int(self)) % self.p ** self.k, self.p, self.k, self.n)
    
    def __pow__(self, n):
        assert(isinstance(n, int) or n.is_integer())
        if n == 0:
            return PAdic(1, self.p, self.k, 0)
        elif n % 2 == 0:
            root_2_res = self ** (n / 2)
            return root_2_res * root_2_res
        else:
            return self * (self ** (n - 1))

In [13]:
oParticles = Particles(6, rational=True)

In [14]:
p = 2 ** 31 - 1
k = 2
n = 0

for oParticle in oParticles:
    oParticle._r_sp_d = numpy.array([PAdic(random.randrange(0, 200), p, k, n), PAdic(random.randrange(0, 200), p, k, n)], dtype=PAdic)
    oParticle._r_sp_d.shape = (2, 1)
    oParticle._r_sp_d_to_r_sp_u()
    oParticle._four_mom = numpy.array([None, None, None, None])
    oParticle.l_sp_d = numpy.array([PAdic(random.randrange(0, 200), p, k, n), PAdic(random.randrange(0, 200), p, k, n)], dtype=PAdic)

oParticles.fix_mom_cons()

In [15]:
# oParticles.set("⟨2|(1+6)|5]", PAdic(p, p, k, n))
oParticles.set("⟨1|2⟩", PAdic(p, p, k, n))

True

In [16]:
oParticles.compute("⟨1|2⟩")

1*2147483647 + O(2147483647^2)

In [17]:
print(oParticles.total_mom)
for i in range(len(oParticles)):
    print(oParticles.ldot(i + 1, i + 1))

[[O(2147483647^2) O(2147483647^2)]
 [O(2147483647^2) O(2147483647^2)]]
O(2147483647^2)
O(2147483647^2)
O(2147483647^2)
O(2147483647^2)
O(2147483647^2)
O(2147483647^2)


In [18]:
oInvariants = Invariants(6)
for oInv in oInvariants.full:
    assert(type(oParticles.compute(oInv)) == PAdic)

In [19]:
# some definitions
zab2_4231 = oParticles.compute("⟨4|(2+3)|1]")
zb_16 = oParticles.compute("[1|6]")
za_23 = oParticles.compute("⟨2|3⟩")
za_34 = oParticles.compute("⟨3|4⟩")
zb_56 = oParticles.compute("[5|6]")
zab2_2165 = oParticles.compute("⟨2|(1+6)|5]")
s_234 = oParticles.compute("s_234")
zab2_6123 = oParticles.compute("⟨6|(1+2)|3]")
za_12 = oParticles.compute("⟨1|2⟩")
za_16 = oParticles.compute("⟨1|6⟩")
zb_34 = oParticles.compute("[3|4]")
zb_45 = oParticles.compute("[4|5]")
s_345 = oParticles.compute("s_345")

In [20]:
mIA6g_pppmmm = zab2_4231 ** 3 / (zb_16 * za_23 * za_34 * zb_56 * zab2_2165 * s_234) - zab2_6123 ** 3 / (za_12 * za_16 * zb_34 * zb_45 * zab2_2165 * s_345)

In [21]:
zab2_4231 ** 3 / (zb_16 * za_23 * za_34 * zb_56 * zab2_2165 * s_234)

40992594 + 1317218337*2147483647 + O(2147483647^2)

In [22]:
- zab2_6123 ** 3 / (za_12 * za_16 * zb_34 * zb_45 * zab2_2165 * s_345)

1342329002*2147483647^-1 + 1984102072 + O(2147483647^1)

In [23]:
mIA6g_pppmmm

1342329002*2147483647^-1 + 2025094666 + O(2147483647^1)

### Subclassing numbers.Rational ?

In [24]:
class rational_subclass(numbers.Rational):
    
    # __slots__ = ('_numerator', '_denominator')
    
    def __new__(cls):
        self = super(rational_subclass, cls).__new__(cls)
        
        self._numerator = 1
        self._denominator = 31
        
        return self
    
    @property
    def numerator(self):
        return self._numerator
    
    @property
    def denominator(self):
        return self._denominator
    
    def __abs__(self):
        pass
    
    def __add__(self, other):
        pass
    
    def __div__(self, other):
        pass
    
    def __eq__(self, other):
        pass
    
    def __floordiv__(self, other):
        pass
    
    def __le__(self, other):
        pass
    
    def __lt__(self, other):
        pass
    
    def __mod__(self, other):
        pass
    
    def __mul__(self, other):
        pass
    
    def __neg__(self, other):
        pass
    
    def __pos__(self, other):
        pass
    
    def __pow__(self, other):
        pass
    
    def __radd__(self, other):
        pass
    
    def __rdiv__(self, other):
        pass
    
    def __rfloordiv__(self, other):
        pass
    
    def __rmod__(self, other):
        pass
    
    def __rmul__(self, other):
        pass
    
    def __rpow__(self, other):
        pass
    
    def __rtruediv__(self, other):
        pass
    
    def __truediv__(self, other):
        pass
    
    def __trunc__(self, other):
        pass